
# NYC Food Access — Geospatial Starter Notebook



In [1]:
import pandas as pd, json
from rich import print

In [2]:
CENSUS_TRACTS_PATH = "data/census_tracts.csv"   # e.g., 2020 Census tracts GeoJSON/GeoPackage/SHAPE
PANTRIES_PATH      = "data/pantry_locations.csv"    # can be a GeoJSON of points OR a CSV with lat/lon columns
OPTIONAL_NEIGHBORHOODS_PATH = None                      # e.g., neighborhoods or NTA boundaries (GeoJSON), or None
SUPPLY_GAP_CSV     = "data/Emergency_Food_Supply_Gap_20251110.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None


In [3]:
tracts = pd.read_csv(CENSUS_TRACTS_PATH)
supply_gap = pd.read_csv(SUPPLY_GAP_CSV)

In [4]:
# Rename for consistency
supply_gap = supply_gap.rename(columns={
    "Neighborhood Tabulation Area NTA)": "nta2020",
    "Neighborhood Tabulation Area (NTA) Name": "ntaname"
})

# Strip spaces / make uppercase (to ensure a clean join)
supply_gap["nta2020"] = supply_gap["nta2020"].str.strip().str.upper()
tracts["nta2020"] = tracts["nta2020"].astype(str).str.strip().str.upper()

In [5]:
supply_gap.head()

,Year,nta2020,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank
0,2025,BX0401,Concourse-Concourse Village,"-102,142.62426294",26.29%,1053.82%,0.57,5.133096,161.0
1,2025,BX0303,Crotona Park East,"-333,492.64867191",27.24%,1308.90%,0.55,5.628794,141.0
2,2025,MN1102,East Harlem (North),"527,499.52610583",26.68%,1055.74%,0.47,5.908840,121.0
3,2025,BX0602,Tremont,"-113,652.81465869",28.96%,1214.11%,0.53,5.711963,134.0
4,2025,BK1204,Mapleton-Midwood (West),"451,455.520860002",16.87%,564.74%,0.61,5.731608,132.0


In [6]:
merged = tracts.merge(supply_gap, on="nta2020", how="left")

In [7]:
merged.dropna(subset=["Supply Gap (lbs.)"], inplace=True)

In [8]:
merged.to_csv("data/neighborhood_supply_gap.csv", index=False)